## Ejercicio - Regresión Viviendas Barcelona

- Utilizando el dataset de publicaciones de venta de viviendas en Barcelona _**viviendas_barcelona.parquet**_, entrena un modelo de regresión lineal múltiple que sea capaz de calcular el precio de venta:

    - Antes de crear el modelo y entrenarlo, haz un **EDA** de los datos.
        - ¿Qué columnas pueden ser las más importantes?
        - ¿Como se distribuyen los precios?
        - ¿Qué columnas sería factible limpiar?
        - ¿Qué significan los **np.nan**?
     
    - Cuando entiendas bien los datos con los que estás tratando, puedes proceder a realizar un preprocesamiento:
        - Elimina duplicados.
        - Elimina columnas innecesarias
        - Trata los valores perdidos **np.nan**.
        - Maneja los outliers como consideres.
        - Realiza transformaciones a los datos que consideres necesarias.
        - Codifica los datos categóricos para que el modelo los entienda (pásalos a números).
        - Considera normalizar los datos antes de utilizarlos para el entreno.
        
    - Entrena el modelo con un subconjunto de 70-80% del dataset.
      
    - Utiliza los datos restantes para comprobar cómo de bueno es el modelo:
        - Calcula **R2**, **MAE** y **MSE**.
        
- Importa las librerías que vayas a necesitar.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelo
from sklearn.linear_model import LinearRegression

# Herramientas de preprocesamiento
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, TargetEncoder, OneHotEncoder

# Split
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df = pd.read_parquet("../Data/viviendas_barcelona.parquet")

In [2]:
df.shape

(27479, 25)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27479 entries, 0 to 27478
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price                  26969 non-null  float64
 1   title                  27129 non-null  object 
 2   province               27479 non-null  object 
 3   location               26873 non-null  object 
 4   lat                    26873 non-null  float64
 5   lng                    26873 non-null  float64
 6   agency                 27129 non-null  object 
 7   updated                27129 non-null  float64
 8   timestamp              27479 non-null  float64
 9   id_1                   27479 non-null  object 
 10  id_2                   27479 non-null  object 
 11  Superficie construida  26978 non-null  object 
 12  Emisiones              16335 non-null  object 
 13  Clasificación          27129 non-null  object 
 14  Baños                  26185 non-null  object 
 15  Co

In [4]:
df.describe(include='all').T
# Vemos que el parametro y sera la columna price y el X el resto 

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
price,26969.0,NaN,NaN,NaN,425521.191071,569359.251647,15500.0,178000.0,270500.0,450000.0,19000000.0
title,27129,12706,Piso en venta en Centre,483,NaN,NaN,NaN,NaN,NaN,NaN,NaN
province,27479,1,barcelona,27479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location,26873,8467,Castellar del Vallès,292,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lat,26873.0,NaN,NaN,NaN,41.501189,0.299589,0.0,41.387962,41.494441,41.576186,42.283257
lng,26873.0,NaN,NaN,NaN,2.122537,0.223253,0.0,2.018452,2.130845,2.218178,3.81478
agency,27129,1329,Inmuebles de Engel & Völkers Barcelona,2187,NaN,NaN,NaN,NaN,NaN,NaN,NaN
updated,27129.0,NaN,NaN,NaN,1673576703.102952,3085560.077187,1579389000.0,1673392200.0,1673996700.0,1674169800.0,1675120080.0
timestamp,27479.0,NaN,NaN,NaN,1547226005864556.5,458334075217092.875,169789023054.0,1697790317799740.0,1697840050993678.0,1697888434903166.0,1697946906943147.0
id_1,27479,27479,34241677924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.columns

Index(['price', 'title', 'province', 'location', 'lat', 'lng', 'agency',
       'updated', 'timestamp', 'id_1', 'id_2', 'Superficie construida',
       'Emisiones', 'Clasificación', 'Baños', 'Consumo', 'Habitaciones',
       'Referencia', 'Antigüedad', 'Garaje', 'Terraza', 'Se aceptan mascotas',
       'Ascensor', 'Tipo de casa', 'Superficie útil'],
      dtype='object')

In [ ]:
# Posibles columnas a descartar
# 'id_1', 'id_2', 'updated', 'timestamp', 'province', 'location', 'Referencia'
df[['id_1', 'id_2', 'updated', 'timestamp', 'province', 'location', 'Referencia']]

* EDA